In [23]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc
from modified_spectral_method import *
from modified_louvain_method import *
from proxy_methods_final import *
import cvxpy as cp
import copy
import matplotlib.pyplot as plt

In [25]:
metadata = pd.read_csv('data/metadata.csv')
reshaped_data = pd.read_csv('data/reshaped_data.csv')

In [27]:
reshaped_data

,Date,AAUK,ABE,ABHLTD,ACAFP,ACAFP-CIB,ACCOR,ACEA,ADIG,AEGON,...,VLVY-Treasury,VOD,VW,WEINBG,WENL,WINDAQ,WOLKLU,WPPGRP-2005,YORPOW,ZINCO
0,2010-09-14 00:00:00,179.666667,252.500000,248.225806,123.000000,120.065811,133.333333,78.555937,89.086828,165.166667,...,143.703161,78.653530,98.287293,312.144549,427.868852,550.204082,68.833333,132.833333,51.549539,99.000000
1,2010-09-15 00:00:00,175.666667,253.553505,250.322581,124.166667,125.346061,133.333333,78.674166,89.746817,166.500000,...,137.526004,80.870588,100.372671,315.809361,422.270270,542.040816,69.011765,132.275229,51.615352,99.500000
2,2010-09-16 00:00:00,174.666667,253.478186,253.658537,122.000000,122.069897,129.833333,77.415326,89.751163,163.333333,...,167.919307,81.562500,97.666667,316.897843,414.516129,556.744186,68.112308,130.788991,51.384026,100.822781
3,2010-09-17 00:00:00,169.500000,253.417889,249.268293,124.000000,122.399586,122.666667,74.098890,89.751278,159.333333,...,175.640643,84.169014,97.225673,317.472295,410.655738,546.081081,69.061224,129.698630,51.399602,95.131222
4,2010-09-20 00:00:00,171.833333,250.460282,254.047183,129.166667,122.810584,126.750000,76.515151,89.060149,162.666667,...,134.164354,81.484536,96.066253,316.516258,415.573770,550.378378,68.108108,132.833333,51.536094,101.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,2020-04-28 00:00:00,176.334600,120.474900,169.815900,54.959700,52.345400,239.073900,88.206400,81.069800,133.937300,...,76.779800,91.104700,164.174700,368.918800,95.373100,41.281800,31.033200,155.939900,75.349900,42.449000
2511,2020-04-29 00:00:00,176.605000,120.885000,170.533200,53.745800,51.238500,237.998100,88.213700,81.057500,134.345000,...,76.791500,90.488700,160.994100,370.851600,95.206200,41.330800,30.780900,148.449200,75.359500,40.402900
2512,2020-04-30 00:00:00,166.690600,120.516700,169.772900,52.172800,50.434200,232.724500,88.213400,81.036100,132.761600,...,76.404700,85.346200,158.997200,369.247500,91.609200,41.309200,29.594200,144.818300,75.353800,39.661100
2513,2020-05-01 00:00:00,166.130300,121.650700,171.038900,54.654500,48.042600,234.903000,88.228100,81.044200,133.690100,...,76.034600,87.044200,165.180000,374.743400,99.308600,41.496100,29.636700,146.774900,75.364800,42.251900


### (i)
Remove all ‘CCC-’ and ‘D’-rated entities since their
spreads contain a large percentage of outliers which
makes our analysis inaccurate. Furthermore, for these
close-to-default entities, banks might decide to use a special methodology instead of the standard proxy method.

In [30]:
# Remove all ratings below CCC- ( AverageRating > 19 )
filtered_metadata = metadata[metadata['AverageRating'] < 19]

# Extract the tickers from the metadata
valid_tickers = filtered_metadata['Ticker']

# Filter the reshaped_data to only keep columns with tickers in metadata, including the "Date" column
filtered_reshaped_data = reshaped_data[['Date'] + [col for col in reshaped_data.columns if col in valid_tickers.values]]

In [32]:
filtered_reshaped_data.shape

(2515, 317)

### (iv)
Remove the quotes from the ‘Government’ sector since
sovereign CDS’s have been studied independently in
the literature

In [35]:
# Remove "Government" sector tickers from filtered_metadata
filtered_metadata = filtered_metadata[filtered_metadata['Sector'] != 'Government']

# Update the list of valid tickers
valid_tickers = filtered_metadata['Ticker']

# Filter the reshaped_data again to only keep columns with tickers in the updated metadata, including the "Date" column
filtered_reshaped_data = reshaped_data[['Date'] + [col for col in reshaped_data.columns if col in valid_tickers.values]]


In [37]:
filtered_reshaped_data.shape

(2515, 297)

### (vii)
Remove quotes for entities which are above 1000 basis
points. This allows us to have a more accurate estimate of
errors in proxying, and similar to ‘CCC-’ and ‘D’-rated
entities, banks might decide to use a special methodology instead of the standard proxy method for these
entities.

In [40]:
# Remove quotes for entities with values above 1000 basis points in the reshaped data
# Excluding the 'Date' column, apply the filtering condition
filtered_reshaped_data = filtered_reshaped_data.copy()
for column in filtered_reshaped_data.columns[1:]:  # Exclude 'Date'
    filtered_reshaped_data[column] = filtered_reshaped_data[column].where(filtered_reshaped_data[column] < 1000)

# Count the total number of NaN values in the filtered dataset
# ( Values above 1000 are turned into NaNs )
nan_count = filtered_reshaped_data.isna().sum().sum()

nan_count

6426

In [42]:
# Remove columns that contain any NaN values
filtered_reshaped_data = filtered_reshaped_data.dropna(axis=1)

In [44]:
filtered_reshaped_data.shape

(2515, 270)

In [50]:
filtered_metadata['Sector'].value_counts()

Sector
Financials            87
Consumer Services     41
Industrials           37
Utilities             34
Consumer Goods        34
Basic Materials       26
Telecommunications    17
Health Care            7
Oil & Gas              7
Technology             6
Name: count, dtype: int64

###
Redefine the sectors Basic Materials, Consumer Services, Energy, Technology, Telecommunication services
and Industrials as Cyclical, and Consumer Goods, Health
care and Utilities as Non Cyclical

In [52]:
# Redefine sectors in metadata
cyclical_sectors = ['Basic Materials', 'Consumer Services', 'Energy', 'Oil & Gas', 'Technology', 
                    'Telecommunications', 'Industrials']
non_cyclical_sectors = ['Consumer Goods', 'Health Care', 'Utilities']

filtered_metadata['Sector'] = np.where(filtered_metadata['Sector'].isin(cyclical_sectors), 'Cyclical', filtered_metadata['Sector'])
filtered_metadata['Sector'] = np.where(filtered_metadata['Sector'].isin(non_cyclical_sectors), 'Non Cyclical', filtered_metadata['Sector'])

# Display the updated metadata DataFrame
filtered_metadata


,Ticker,ShortName,Sector,Region,Country,AverageRating
0,AAUK,Anglo Amern plc,Cyclical,Europe,United Kingdom,14
1,ABE,Abertis Infraestructuras SA,Cyclical,Europe,Spain,9
2,ABHLTD,Alliance Boots Hldgs Ltd,Cyclical,Europe,United Kingdom,10
3,ACAFP,Cr Agricole SA,Financials,Europe,France,5
4,ACAFP-CIB,Cr Agricole Corporate and Invt Bk,Financials,Europe,France,10
...,...,...,...,...,...,...
313,WINDAQ,Wind Acquisition Fin SA,Financials,Europe,Luxembourg,17
314,WOLKLU,Wolters Kluwer N V,Cyclical,Europe,Netherlands,9
315,WPPGRP-2005,WPP 2005 Ltd,Cyclical,Europe,United Kingdom,17
316,YORPOW,Yorkshire Pwr Gp Ltd,Non Cyclical,Europe,United Kingdom,10


In [54]:
filtered_metadata['Sector'].value_counts()

Sector
Cyclical        134
Financials       87
Non Cyclical     75
Name: count, dtype: int64

In [17]:
# Extract the list of tickers from the final filtered reshaped data (excluding the 'Date' column)
final_tickers = set(filtered_reshaped_data.columns[1:])

# Filter the metadata to include only tickers that are in the final reshaped data
filtered_metadata_final = filtered_metadata[filtered_metadata['Ticker'].isin(final_tickers)]

# Display the filtered metadata
print(filtered_metadata_final)


          Ticker                          ShortName        Sector  Region  \
1            ABE        Abertis Infraestructuras SA      Cyclical  Europe   
2         ABHLTD           Alliance Boots Hldgs Ltd      Cyclical  Europe   
3          ACAFP                     Cr Agricole SA    Financials  Europe   
4      ACAFP-CIB  Cr Agricole Corporate and Invt Bk    Financials  Europe   
5          ACCOR                              ACCOR      Cyclical  Europe   
..           ...                                ...           ...     ...   
312         WENL                             WENDEL    Financials  Europe   
314       WOLKLU                 Wolters Kluwer N V      Cyclical  Europe   
315  WPPGRP-2005                       WPP 2005 Ltd      Cyclical  Europe   
316       YORPOW               Yorkshire Pwr Gp Ltd  Non Cyclical  Europe   
317        ZINCO                  Zurich Ins Co Ltd    Financials  Europe   

            Country  AverageRating  
1             Spain              9  
2

In [21]:
# Export the final filtered metadata to a CSV file
filtered_metadata_final.to_csv('data/new_metadata.csv', index=False)

# Export the final filtered reshaped data to a CSV file
filtered_reshaped_data.to_csv('data/new_reshaped_data.csv', index=False)